In [46]:
!pip3 install lxml
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [77]:
#find links to each person

link = 'https://time.com/collection/100-most-influential-people-2020/'


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response2020 = requests.get(link, headers=headers)
soup2020 = BeautifulSoup(response2020.content)
soup2020.prettify()
lists = soup2020.findAll("ul", {"class":"full-list-content"})

categories = []
urls2020 = []

#docs
for l in lists:   
    cut = re.findall(r'</?a(|\s+[^>]+)>',str(l))
    for link in cut:
        if link != '':
            urls2020.append(link[7:-1])

urls2020



    

['https://time.com/collection/100-most-influential-people-2020/5888165/megan-thee-stallion-pioneer/',
 'https://time.com/collection/100-most-influential-people-2020/5888173/giannis-antetokounmpo/',
 'https://time.com/collection/100-most-influential-people-2020/5888207/ibram-x-kendi/',
 'https://time.com/collection/100-most-influential-people-2020/5888201/nathan-law/',
 'https://time.com/collection/100-most-influential-people-2020/5888211/tomi-adeyemi/',
 'https://time.com/collection/100-most-influential-people-2020/5888216/all-women-spacewalk-christina-koch-jessica-meir/',
 'https://time.com/collection/100-most-influential-people-2020/5888209/julie-k-brown/',
 'https://time.com/collection/100-most-influential-people-2020/5888195/cecilia-martinez/',
 'https://time.com/collection/100-most-influential-people-2020/5888213/maya-moore/',
 'https://time.com/collection/100-most-influential-people-2020/5888158/chase-strangio/',
 'https://time.com/collection/100-most-influential-people-2020/5888

In [92]:
'''
for line in headdiv:
    person = line.findAll('li')
    people.append(person[7:-1])
people
'''
#2019 test
link2019 = 'https://time.com/collection/100-most-influential-people-2019/'
response2019 = requests.get(link2019, headers=headers)
soup2019 = BeautifulSoup(response2019.content)
soup2019.prettify()
#headdiv = soup2019.findAll("div", {"class": "C_ClVyao"})
people = soup2019.findAll('ul')



In [70]:
def getDescription(url):
    '''
    Takes in a url to any specific person and returns their description
    '''
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    #use Beautiful Soup to prettify the content
    soup = BeautifulSoup(response.content)
    soup.prettify()
    
    #retrieve paragraph text
    articleBody = soup.find("div", {"class": "article content body clearfix"}).get_text().strip('\n')
    comment = articleBody.split('\n\n\n\n\n')[0]
    
    #determine gender based on first pronoun found??? need better way
    if 'she' in comment:
        gender = 'Female'
    elif 'he' in comment:
        gender = 'Male'
    else:
        gender = ''
    
    return comment, gender
    

In [71]:
# test with Queen Meg <3 <3
url = 'https://time.com/collection/100-most-influential-people-2020/5888165/megan-thee-stallion-pioneer/'
getDescription(url)

('I remember hearing Megan Thee Stallion on one of those famous DJ radio shows a few years ago. She rode the beat like I’d never heard anybody ride the beat in a long time—and I’m a hip-hop head. There was something about this woman. Once you discover her, you become a fan. I don’t like to put the stigma of the word strong on Black women because I think it dehumanizes us, but she has strength—strength through vulnerability. She’s lost much of her family—her mother, her father, her grandmother—yet she is the epitome of tenacity, of pulling herself up by her bootstraps. She was shot this summer, and still people tried to tear her down. But she’s out here still loving and being sweet. It’s invigorating to see her become a platinum-selling artist with the viral hit “Hot Girl Summer” and multiple No. 1 songs in the past year, “Savage” and “WAP.” But you would be a fool to think that’s all there is to her. She’s deep. She’s enrolled in college. She’s an entertainer. She’s a free spirit; I se

In [72]:
comments = {}

#run for all of them
for url in urls2020:
    comment, gender = getDescription(url)
    #add to dictionary
    comments[comment] = gender

In [73]:
commentsByGender = pd.DataFrame(list(comments.items()),columns = ['Comment','Gender'])
commentsByGender

,Comment,Gender
0,I remember hearing Megan Thee Stallion on one ...,Female
1,Paul Simon sang that “every generation throws ...,Male
2,As we march to dramatize the assaults on Black...,Male
3,"Nathan Law, a pro-democracy activist now drive...",Male
4,When someone told me about Tomi Adeyemi’s Chil...,Male
...,...,...
93,"If we stop, the world stops. Those were the wo...",Male
94,"Nury Turkel and I are Uyghurs, part of the maj...",Male
95,Shine the light. That’s the mission of journal...,Male
96,"When I first met Bilkis, she sat in the midst ...",Female


# NLP Analysis

In [52]:
# Vectorize the whole thing...
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Vectorize and play with token sizes...
vec = TfidfVectorizer(min_df = 0.05, 
                      max_df = 0.95, 
                      ngram_range=(1,1)) # play with min_df and max_df

coms = commentsByGender['Comment']

# transform this into a sparse vector!
vec.fit(coms)
tf_idf_sparse = vec.transform(coms)
tf_idf_sparse


<98x540 sparse matrix of type '<class 'numpy.float64'>'
	with 6821 stored elements in Compressed Sparse Row format>

In [53]:
#get list of cleaned, stemmed adjectives, for later analysis. 
vect = CountVectorizer()
vect.fit(coms)
words = vect.get_feature_names()

adjectives = []
stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

In [54]:
print(tf_idf_sparse[0, :])

  (0, 537)	0.14814492748803515
  (0, 535)	0.080332861431985
  (0, 534)	0.053400724817117357
  (0, 533)	0.04795305923741796
  (0, 531)	0.061803195468619634
  (0, 523)	0.061803195468619634
  (0, 522)	0.07305622881377255
  (0, 520)	0.03012652227927581
  (0, 519)	0.05522969185563041
  (0, 504)	0.03670002589226504
  (0, 500)	0.06354221760845938
  (0, 498)	0.080332861431985
  (0, 484)	0.053400724817117357
  (0, 483)	0.04472942883364162
  (0, 462)	0.04299315924340028
  (0, 460)	0.04938164249601172
  (0, 457)	0.06056776542831353
  (0, 456)	0.11676062047045861
  (0, 452)	0.11879143862241114
  (0, 447)	0.07900134060783755
  (0, 434)	0.16066572286397
  (0, 431)	0.08690636504497422
  (0, 430)	0.16676807803948696
  (0, 425)	0.1461124576275451
  (0, 417)	0.080332861431985
  :	:
  (0, 223)	0.08690636504497422
  (0, 222)	0.5247298908077899
  (0, 217)	0.080332861431985
  (0, 212)	0.029500458742251077
  (0, 203)	0.13847812282357852
  (0, 184)	0.07764153206329155
  (0, 168)	0.08338403901974348
  (0, 163)

In [55]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    commentsByGender['Gender'], 
                                                    test_size=0.2)

In [56]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

/Users/laurensussman/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [57]:
logisticRegr.classes_

array(['Female', 'Male'], dtype=object)

In [58]:
logisticRegr.coef_[:,0]

array([0.07421779])

In [59]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [61]:
## uncomment next line to see non filtered Male results
#df_weights.sort_values('weight', ascending=False)[:25]
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
273,environmental,0.340484
225,know,0.242743
152,important,0.199689
210,human,0.198614
388,iconic,0.156717
362,live,0.155959
428,political,0.132228
148,new,0.129881
80,true,0.125736
306,united,0.116051


In [62]:
## uncomemnt for female unfiltered results
#df_weights.sort_values('weight', ascending=True)[:25]
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
femaleAdjectives

,word,weight
260,trump,-0.335956
240,female,-0.225346
38,much,-0.224727
95,possible,-0.200816
323,pandemic,-0.198748
324,global,-0.173606
28,black,-0.159299
382,economic,-0.143649
385,visionary,-0.137859
195,last,-0.127268


In [63]:
#verify results
agentic_terms = ['assertive','confident','aggressive','ambitious','dominan','forceful','independent','daring','outspoken','intellectua','earn','gain','know','insight','think']
communal_terms = ['sympathetic','kind','help','affection','sensitive','nurtur','agreeab','tactful','interpersonal','warm','car','tactful']
socio_communal = ['husband','wife','kid','babies','brother','child','colleague','family']

print('Results for most commonly found female adjectives:')
print('Agentic: ', femaleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', femaleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', femaleAdjectives['word'].isin(socio_communal).sum())
print('')
print('Results for most commonly found male adjectives:')
print('Agentic: ', maleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', maleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', maleAdjectives['word'].isin(socio_communal).sum())

Results for most commonly found female adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0

Results for most commonly found male adjectives:
Agentic:  1
Communal:  0
Socio-communal:  0
